# Visualizations - Telegram

In [ ]:
%pip install trino==0.332.0 trino[sqlalchemy] seaborn polars

# Step 1 - Preliminary settings

In [ ]:
import trino.dbapi
import os 
import pandas as pd
import polars as pl
import datetime as dt
import matplotlib.pyplot as plt

In [ ]:
TRINO_HOST=os.environ.get("TRINO_HOST")
TRINO_USER=os.environ.get("TRINO_USER")

In [ ]:
def df_from_db(query, col_names=None, host=TRINO_HOST, user=TRINO_USER, catalog='iceberg', **df_kwargs):
    with trino.dbapi.connect(host=host, user=user, catalog=catalog) as conn:
        df = df_from_db_conn(query, conn, col_names, **df_kwargs)
    return df

def df_from_db_conn(query, conn, col_names=None, **df_kwargs):
    cur = conn.cursor()
    res = cur.execute(query).fetchall()
    cur.close()
    df = pl.DataFrame(res, orient='row', **df_kwargs)
    if col_names is not None:
        df = df.rename({f"column_{i}": c for i, c in enumerate(col_names)})
    return df

## Content reliability

In [ ]:
query = "select * from news.lasser_dq_scores"
domains = df_from_db(query, ['domain', 'reliability'], catalog='db')

In [ ]:
dt_range = pl.datetime_range(dt.date(2025, 1, 1), dt.date.today() - dt.timedelta(days=2))

query = """
with date_count as (
    select id, cast(from_iso8601_timestamp(date) as DATE) as date
    from telegram.messages
)
select date, count(*) from date_count group by date
"""
# select cast(from_iso8601_timestamp(date) as DATE) as date_only, count(*) from telegram.messages group by date_only
df = df_from_db(query, ['date', 'message_count'])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(x='date', height='message_count', data=df, color='skyblue')
ax.set_xlabel('Date')
ax.set_ylabel('Number of messages')
fig.show()

## Check tools output for TG

In [ ]:
with trino.dbapi.connect(host=TRINO_HOST, user=TRINO_USER, catalog='iceberg') as dc_conn:
    cur = dc_conn.cursor()
    a = cur.execute("select * from tools.text_disinformation_signals_detection limit 5").fetchall()

In [ ]:
a[0][3][0]